In [3]:
pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 26.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 25.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 32.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [28]:
import pandas as pd
import os
import json
# DIRECTORY = "E:\\research\\projects\\teamwork-visualiser-dashboard\\server\\saved_data\\"
DIRECTORY = "/Users/riordanalfredo/Desktop/research-softeng/teamwork-visualiser-dashboard/server/saved_data"


In [33]:

def read_csv_by_time(file_path, start_time_int, end_time_int):
    df = pd.read_csv(file_path)

    # Filter the DataFrame based on the given start_time and end_time
    filtered_df = df[(df['seconds_int'] >= start_time) & (df['seconds_int'] <= end_time)]
    
    return filtered_df


start_time = 0
end_time = 3000
id = "374"

file_new = "%s_coteach.csv" % id
# file_path = DIRECTORY / id / "result" / file
file_path = os.path.join(DIRECTORY, id, "result", file_new)


data = read_csv_by_time(file_path, start_time, end_time)
data.head()


,seconds_int,starting_time,TA,nan,surveillance,assisting,monitoring,1-1 student teacher interaction,lecturing,personal,...,BG_parallel-teaching,BG_None,RG_coteaching,RG_team-teaching,RG_alternative-teaching,RG_one-teach-one-helps,RG_one-teach-one-observes,RG_parallel-teaching,RG_None,time
0,0,2024-03-25 14:09:05.080,B,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,00:00:00
1,0,2024-03-25 14:09:05.080,G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,00:00:00
2,0,2024-03-25 14:09:05.080,R,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,00:00:00
3,1,2024-03-25 14:09:05.080,B,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,00:00:01
4,1,2024-03-25 14:09:05.080,G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,00:00:01


In [13]:
pedagogy_map = {
        "authoritative": ["lecturing", "assisting"],
        "supervisory": ["monitoring", "surveillance"],
        "interactional": ["1-1 student teacher interaction"],
        "personal": ["personal", "watching"]
    }

In [8]:
def count_pedagogies_by_ta(df, colour_symbol:str):
    
    ta_filtered_df = df[df['TA'] == colour_symbol]
    # Drop columns that are not activity related
    ped_columns = ['surveillance', 'assisting', 'monitoring', '1-1 student teacher interaction', 'teacher-teacher interaction', 'lecturing', 'personal', 'watching']
    
    # Count the occurrences of each activity
    ped_counts = ta_filtered_df[ped_columns].sum()

    # Calculate the total counts of activities
    total_counts = ped_counts.sum()
    
    # Calculate percentages
    activity_percentages = (ped_counts / total_counts) * 100
    
    return activity_percentages


ped_counts = count_pedagogies_by_ta(data, 'B')
print(ped_counts)

surveillance                        0.000000
assisting                           1.332889
monitoring                         31.589470
1-1 student teacher interaction     6.131290
teacher-teacher interaction         0.000000
lecturing                          38.487171
personal                           14.561813
watching                            7.897368
dtype: float64


In [9]:
def map_activities_to_arrays(ped_counts):
    
    # Flatten the pedagogy map into two arrays based on the index positions
    primary = ['lecturing', 'monitoring', '1-1 student teacher interaction', 'personal']
    secondary = ['assisting', 'surveillance', 'teacher-teacher interaction', 'watching']
    
    # Initialize the arrays to store the counts
    first_array_counts = []
    second_array_counts = []
    
    # Populate the arrays with the counts from ped_counts
    for activity in primary:
        first_array_counts.append(round(ped_counts.get(activity, 0)))
    
    for activity in secondary:
        second_array_counts.append(round(ped_counts.get(activity, 0)))
    
    return first_array_counts, second_array_counts

first_array_counts, second_array_counts = map_activities_to_arrays(ped_counts)
res_by_person = [first_array_counts, second_array_counts]
res_by_person



[[38, 32, 6, 15], [1, 0, 0, 8]]

In [10]:
def map_activities_to_json(activity_counts):
    # Create a dictionary to store the mapped activity counts
    pedagogy_counts = {}
    
    for pedagogy, activities in pedagogy_map.items():
        pedagogy_counts[pedagogy] = {}
        for activity in activities:
            pedagogy_counts[pedagogy][activity] = round(activity_counts.get(activity, 0))
    
    return pedagogy_counts

pedagogy_counts = map_activities_to_json(ped_counts)


pedagogy_counts_json = json.dumps(pedagogy_counts)
pedagogy_counts_json

# pedagogy_counts_json

'{"authoritative": {"lecturing": 38, "assisting": 1}, "supervisory": {"monitoring": 32, "surveillance": 0}, "interactional": {"1-1 student teacher interaction": 6}, "personal": {"personal": 15, "watching": 8}}'

# get all matrix

In [14]:
def calculate_percentage(df, pedagogy_filter, teacher_filter):

    ta_filtered_df = df[df['TA'] == teacher_filter]

    ped_columns = pedagogy_map[pedagogy_filter]
    
    
    # Count the occurrences of each activity
    ped_counts = ta_filtered_df[ped_columns].sum()

    # # Calculate the total counts of activities
    total_counts = ped_counts.sum()
    
    # # Calculate percentages
    activity_percentages = round((ped_counts / (total_counts + 1e-10)) * 100)
    
    # return total_counts
    return activity_percentages.to_dict()
            

pedagogy = ['authoritative', 'supervisory', 'interactional', 'personal']
ta_colours = ['R', 'G', 'B']

result = {}
for ped in pedagogy:
    result[ped] = {}
    for ta in ta_colours:
        result[ped][ta] = calculate_percentage(data, ped, ta)

result


{'authoritative': {'R': {'lecturing': 100.0, 'assisting': 0.0},
  'G': {'lecturing': 0.0, 'assisting': 0.0},
  'B': {'lecturing': 97.0, 'assisting': 3.0}},
 'supervisory': {'R': {'monitoring': 100.0, 'surveillance': 0.0},
  'G': {'monitoring': 75.0, 'surveillance': 25.0},
  'B': {'monitoring': 100.0, 'surveillance': 0.0}},
 'interactional': {'R': {'1-1 student teacher interaction': 100.0},
  'G': {'1-1 student teacher interaction': 100.0},
  'B': {'1-1 student teacher interaction': 100.0}},
 'personal': {'R': {'personal': 19.0, 'watching': 81.0},
  'G': {'personal': 57.0, 'watching': 43.0},
  'B': {'personal': 65.0, 'watching': 35.0}}}

In [40]:
def calc_all_by_teacher_raw(df, teacher_filter):
    ped_columns = ['surveillance', 'assisting', 'monitoring', '1-1 student teacher interaction', 'teacher-teacher interaction', 'lecturing', 'personal', 'watching']

    ta_filtered_df = df[df['TA'] == teacher_filter]

    # Count the occurrences of each activity
    ped_counts = ta_filtered_df[ped_columns].sum()

    return ped_counts.to_dict()
            

# pedagogy = ['authoritative', 'supervisory', 'interactional', 'personal']
ta_colours = {"RED": 'R', "GREEN": 'G', "BLUE": 'B'}


def get_matrix():
    result = {}
    for ta in ta_colours.keys():
        result[ta] = {}
        calc_data = calc_all_by_teacher_raw(data, ta_colours[ta])
        temp ={}
        for pedagogy, activities in pedagogy_map.items():
            result[ta][pedagogy] = sum(calc_data[activity] for activity in activities if activity in calc_data)
        temp = result[ta][pedagogy] 
        total = sum(result[ta].values())
        temp = {
            pedagogy: round((count / total) * 100) for pedagogy, count in result[ta].items()
        }
        result[ta]=temp
    return result

get_matrix()

{'RED': {'authoritative': 40,
  'supervisory': 9,
  'interactional': 26,
  'personal': 25},
 'GREEN': {'authoritative': 0,
  'supervisory': 15,
  'interactional': 45,
  'personal': 40},
 'BLUE': {'authoritative': 40,
  'supervisory': 32,
  'interactional': 6,
  'personal': 22}}

In [49]:
color_mapping = {
    "RED": "R",
    "GREEN": "G",
    "BLUE": "B"
}

def get_complete_coteach_data(data):
    result = {}
    for ta in color_mapping.keys(): 
        result[ta] = {}
        calc_data = calc_all_by_teacher_raw(data, color_mapping[ta])
        result[ta] = calc_data
        total = sum(result[ta].values())
        temp = {
            pedagogy: round((count / total) * 100) for pedagogy, count in result[ta].items()
        }
        result[ta]=temp
    print(result)
    # Function to convert the data based on the mapping
    res = {category: {} for category in pedagogy_map}
    for color, activities in result.items():
        for category, activity_list in pedagogy_map.items():
            if color not in res[category]:
                res[category][color] = {}
            for activity in activity_list:
                if activity in activities:
                    res[category][color][activity] = activities[activity]
    return res
get_complete_coteach_data(data)

{'RED': {'surveillance': 0, 'assisting': 0, 'monitoring': 8, '1-1 student teacher interaction': 25, 'teacher-teacher interaction': 4, 'lecturing': 39, 'personal': 4, 'watching': 19}, 'GREEN': {'surveillance': 4, 'assisting': 0, 'monitoring': 11, '1-1 student teacher interaction': 42, 'teacher-teacher interaction': 5, 'lecturing': 0, 'personal': 22, 'watching': 16}, 'BLUE': {'surveillance': 0, 'assisting': 1, 'monitoring': 32, '1-1 student teacher interaction': 6, 'teacher-teacher interaction': 0, 'lecturing': 38, 'personal': 15, 'watching': 8}}


{'authoritative': {'RED': {'lecturing': 39, 'assisting': 0},
  'GREEN': {'lecturing': 0, 'assisting': 0},
  'BLUE': {'lecturing': 38, 'assisting': 1}},
 'supervisory': {'RED': {'monitoring': 8, 'surveillance': 0},
  'GREEN': {'monitoring': 11, 'surveillance': 4},
  'BLUE': {'monitoring': 32, 'surveillance': 0}},
 'interactional': {'RED': {'1-1 student teacher interaction': 25},
  'GREEN': {'1-1 student teacher interaction': 42},
  'BLUE': {'1-1 student teacher interaction': 6}},
 'personal': {'RED': {'personal': 4, 'watching': 19},
  'GREEN': {'personal': 22, 'watching': 16},
  'BLUE': {'personal': 15, 'watching': 8}}}